# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 1 2022 8:30AM,245474,10,0085955024,ISDIN Corporation,Released
1,Sep 1 2022 8:30AM,245474,10,0085955025,ISDIN Corporation,Released
2,Sep 1 2022 8:30AM,245474,10,0085955052,ISDIN Corporation,Released
3,Sep 1 2022 8:30AM,245474,10,0085955053,ISDIN Corporation,Released
4,Sep 1 2022 8:30AM,245474,10,0085955059,ISDIN Corporation,Released
5,Sep 1 2022 8:30AM,245474,10,0085955060,ISDIN Corporation,Released
6,Sep 1 2022 8:30AM,245474,10,0085955061,ISDIN Corporation,Released
7,Aug 31 2022 3:36PM,245473,19,8467965,"Graystone, LLC",Released
8,Aug 31 2022 3:16PM,245471,19,ADV80005395,"AdvaGen Pharma, Ltd",Executing
9,Aug 31 2022 3:14PM,245470,10,TXMD-DR-0100.1-BO,TherapeuticsMD Sample,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,245469,Released,2
20,245470,Released,1
21,245471,Executing,1
22,245473,Released,1
23,245474,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245469,NaN,NaN,2.0
245470,NaN,NaN,1.0
245471,NaN,1.0,NaN
245473,NaN,NaN,1.0
245474,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245403,11.0,11.0,0.0
245411,0.0,0.0,1.0
245414,0.0,0.0,1.0
245415,0.0,0.0,1.0
245419,1.0,0.0,29.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245403,11,11,0
245411,0,0,1
245414,0,0,1
245415,0,0,1
245419,1,0,29


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245403,11,11,0
1,245411,0,0,1
2,245414,0,0,1
3,245415,0,0,1
4,245419,1,0,29


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245403,11,11,
1,245411,,,1
2,245414,,,1
3,245415,,,1
4,245419,1,,29


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation
7,Aug 31 2022 3:36PM,245473,19,"Graystone, LLC"
8,Aug 31 2022 3:16PM,245471,19,"AdvaGen Pharma, Ltd"
9,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample
10,Aug 31 2022 2:52PM,245469,10,"Nextsource Biotechnology, LLC"
12,Aug 31 2022 2:41PM,245466,10,ISDIN Corporation
24,Aug 31 2022 2:40PM,245462,10,ISDIN Corporation
54,Aug 31 2022 1:56PM,245454,10,"Senseonics, Incorporated"
55,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd"
73,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,,,7
1,Aug 31 2022 3:36PM,245473,19,"Graystone, LLC",,,1
2,Aug 31 2022 3:16PM,245471,19,"AdvaGen Pharma, Ltd",,1,
3,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample,,,1
4,Aug 31 2022 2:52PM,245469,10,"Nextsource Biotechnology, LLC",,,2
5,Aug 31 2022 2:41PM,245466,10,ISDIN Corporation,,,12
6,Aug 31 2022 2:40PM,245462,10,ISDIN Corporation,,,30
7,Aug 31 2022 1:56PM,245454,10,"Senseonics, Incorporated",,1,
8,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd",,18,
9,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,,2,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7,,
1,Aug 31 2022 3:36PM,245473,19,"Graystone, LLC",1,,
2,Aug 31 2022 3:16PM,245471,19,"AdvaGen Pharma, Ltd",,1,
3,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample,1,,
4,Aug 31 2022 2:52PM,245469,10,"Nextsource Biotechnology, LLC",2,,
5,Aug 31 2022 2:41PM,245466,10,ISDIN Corporation,12,,
6,Aug 31 2022 2:40PM,245462,10,ISDIN Corporation,30,,
7,Aug 31 2022 1:56PM,245454,10,"Senseonics, Incorporated",,1,
8,Aug 31 2022 1:37PM,245445,19,"AdvaGen Pharma, Ltd",,18,
9,Aug 31 2022 1:35PM,245422,20,Alumier Labs Inc.,4,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7,,
1,Aug 31 2022 3:36PM,245473,19,"Graystone, LLC",1,,
2,Aug 31 2022 3:16PM,245471,19,"AdvaGen Pharma, Ltd",,1,
3,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample,1,,
4,Aug 31 2022 2:52PM,245469,10,"Nextsource Biotechnology, LLC",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7.0,NaN,NaN
1,Aug 31 2022 3:36PM,245473,19,"Graystone, LLC",1.0,NaN,NaN
2,Aug 31 2022 3:16PM,245471,19,"AdvaGen Pharma, Ltd",NaN,1.0,NaN
3,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample,1.0,NaN,NaN
4,Aug 31 2022 2:52PM,245469,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 8:30AM,245474,10,ISDIN Corporation,7.0,0.0,0.0
1,Aug 31 2022 3:36PM,245473,19,"Graystone, LLC",1.0,0.0,0.0
2,Aug 31 2022 3:16PM,245471,19,"AdvaGen Pharma, Ltd",0.0,1.0,0.0
3,Aug 31 2022 3:14PM,245470,10,TherapeuticsMD Sample,1.0,0.0,0.0
4,Aug 31 2022 2:52PM,245469,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1963649,57.0,6.0,0.0
15,245419,29.0,0.0,1.0
19,736389,1.0,19.0,0.0
20,490825,4.0,13.0,11.0
21,490829,2.0,0.0,0.0
22,981719,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1963649,57.0,6.0,0.0
1,15,245419,29.0,0.0,1.0
2,19,736389,1.0,19.0,0.0
3,20,490825,4.0,13.0,11.0
4,21,490829,2.0,0.0,0.0
5,22,981719,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,57.0,6.0,0.0
1,15,29.0,0.0,1.0
2,19,1.0,19.0,0.0
3,20,4.0,13.0,11.0
4,21,2.0,0.0,0.0
5,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,57.0
1,15,Released,29.0
2,19,Released,1.0
3,20,Released,4.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,0.0,1.0,0.0,11.0,0.0,0.0
Executing,6.0,0.0,19.0,13.0,0.0,0.0
Released,57.0,29.0,1.0,4.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,0.0,1.0,0.0,11.0,0.0,0.0
1,Executing,6.0,0.0,19.0,13.0,0.0,0.0
2,Released,57.0,29.0,1.0,4.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,0.0,1.0,0.0,11.0,0.0,0.0
1,Executing,6.0,0.0,19.0,13.0,0.0,0.0
2,Released,57.0,29.0,1.0,4.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()